In [0]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import nltk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn import linear_model
import re

In [0]:

from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

In [0]:

from pylab import rcParams
rcParams['figure.figsize'] = 10, 8
sns.set_style('whitegrid')

import sklearn
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn import metrics
from sklearn.metrics import classification_report

import nltk

In [0]:
mbti = pd.read_csv('/content/drive/My Drive/Unsupervised Machine Learning/train.csv')
test = pd.read_csv('/content/drive/My Drive/Unsupervised Machine Learning/test.csv')
mbti.head()

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


In [0]:
all_mbti = pd.concat((mbti[['posts']].copy(),test[['posts']].copy()),axis=0)

In [0]:
all_mbti['posts'] = all_mbti['posts'].apply(lambda x: ' '.join(x.split('|||')))

__Data Cleaning__

_Removing Noise_

_Remove url_

In [0]:
pattern_url = r'http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9A-Fa-f][0-9A-Fa-f]))+'
subs_url = r'url-web'
all_mbti['posts'] = all_mbti['posts'].replace(to_replace = pattern_url, value = subs_url, regex = True)

_To Lowercase_

In [0]:
all_mbti['posts'] = all_mbti['posts'].str.lower()


In [0]:
all_mbti.head()

,posts
0,'url-web url-web enfp and intj moments url-we...
1,'i'm finding the lack of me in these posts ver...
2,"'good one _____ url-web of course, to which..."
3,"'dear intp, i enjoyed our conversation the o..."
4,'you're fired. that's another silly misconcept...


_Remove Punctuation_

In [0]:
import string
def remove_punctuation_numbers(post):
    punc_numbers = string.punctuation + '0123456789'
    return ''.join([l for l in post if l not in punc_numbers])
all_mbti['posts'] = all_mbti['posts'].apply(remove_punctuation_numbers)

__Tokenising__

In [0]:
from nltk.tokenize import word_tokenize, TreebankWordTokenizer

In [0]:
tokeniser = TreebankWordTokenizer()
all_mbti['posts'] = all_mbti['posts'].apply(tokeniser.tokenize)

__Stemming__

In [0]:
from nltk import SnowballStemmer, PorterStemmer, LancasterStemmer

In [0]:
stemmer = SnowballStemmer('english')
def mbti_stemmer(words, stemmer):
    return [stemmer.stem(word) for word in words]

In [0]:
#stem all words in the mbti dataframe
all_mbti['posts'] = all_mbti['posts'].apply(mbti_stemmer, args=(stemmer, ))

__Lemmatization__

In [0]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Mzwandile.m\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
def mbti_lemma(words, lemmatizer):
    return [lemmatizer.lemmatize(word) for word in words]

In [0]:
#lemmatize all words in dataframe
all_mbti['posts'] = all_mbti['posts'].apply(mbti_lemma, args=(lemmatizer, ))

__Stopwords__

In [0]:
from nltk.corpus import stopwords

In [0]:
def remove_stop_words(tokens):
     return [t for t in tokens if t not in stopwords.words('english')]

In [0]:
# def remove_stop_words(tokens):
#     mystopwrords = set(stopwords.words('english'))
#     New_stopwords = []
#     set_of_stopwords=['i','you','we','ours','yourself','her','he','she','herself','my','yours','him','his','them','their']
#     for i in mystopwrords:
#         if i not in set_of_stopwords:
#             New_stopwords.append(i)
            
#     return New_stopwords

In [0]:
# all_mbti['posts'] = all_mbti['posts'].apply(remove_stop_words)

In [0]:
all_mbti['posts'] = [' '.join(line) for line in all_mbti['posts']]

In [0]:
all_mbti

,posts
0,urlweb urlweb enfp and intj moment urlweb spor...
1,im find the lack of me in these post veri alar...
2,good one urlweb of cours to which i say i know...
3,dear intp i enjoy our convers the other day es...
4,your fire that anoth silli misconcept that app...
5,scienc is not perfect no scientist claim that ...
6,no i cant draw on my own nail haha those were ...
7,i tend to build up a collect of thing on my de...
8,im not sure that a good question the distinct ...
9,urlweb im in this posit where i have to actual...


__Create Predictor Variable__

In [0]:
mbti['mind'] = mbti['type'].str.contains('E') * 1
mbti['energy'] = mbti['type'].str.contains('N') * 1
mbti['nature'] = mbti['type'].str.contains('T') * 1
mbti['tactics'] = mbti['type'].str.contains('J')*1

In [0]:
my_train = all_mbti[:len(mbti)]
my_test = all_mbti[len(mbti):]

In [0]:
my_test

,posts
0,music linguist logic naturalist tie genuin que...
1,you hello stranger hii you how are you today s...
2,what work for me wa know that limer in itself ...
3,plea forget about him you should definit save ...
4,ooh entj celestia is actual an interest though...
5,hi dupin im and from new hampshir thank you so...
6,if we are asshol it becaus we suck at fe most ...
7,favorit indiefolk peac side of me rock live si...
8,hello welcom to person cafe they tast differ i...
9,here a short documentari about life a a canadi...


In [0]:
from sklearn.feature_extraction.text import CountVectorizer

In [0]:
vect = CountVectorizer()
X_vect = vect.fit_transform(my_train['posts'])
Test_vect = vect.transform(my_test['posts'])

In [0]:
X_vect.shape

(6506, 133)

In [0]:
logreg = LogisticRegression(C=3,penalty='l1')

In [0]:
X = X_vect
#test_train_split_tactics
X_train_e, X_test_e, y_train_e, y_test_e = train_test_split(X,mbti['energy'].values)
logreg.fit(X_train_e,y_train_e)

y_pred_e = logreg.predict(X_test_e)
metrics.accuracy_score(y_test_e, y_pred_e)


0.858020897357099

In [0]:
#test_train_split_tactics
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(X,mbti['mind'].values)
logreg.fit(X_train_m,y_train_m)
# generate predictions
y_pred_m = logreg.predict(X_test_m)
metrics.accuracy_score(y_test_m, y_pred_m)

0.7566072526121697

In [0]:
#test_train_split_tactics
X_train_n, X_test_n, y_train_n, y_test_n = train_test_split(X,mbti['nature'].values)
logreg.fit(X_train_n,y_train_n)
# generate predictions
y_pred_n = logreg.predict(X_test_n)
metrics.accuracy_score(y_test_n, y_pred_n)

0.5451751690227412

In [0]:
#test_train_split_tactics
X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(X,mbti['tactics'].values)
logreg.fit(X_train_t,y_train_t)
# generate predictions
y_pred_t = logreg.predict(X_test_t)
metrics.accuracy_score(y_test_t, y_pred_t)

0.6189305470190535

In [0]:
sub = test[['id']]

In [0]:
logreg.fit(X_vect,mbti['mind'].values)
y_pred_mind = logreg.predict(Test_vect)
sub['mind'] = y_pred_mind

In [0]:
logreg.fit(X_vect,mbti['energy'].values)
sub['energy'] = logreg.predict(Test_vect)

In [0]:
logreg.fit(X_vect,mbti['nature'].values)
sub['nature'] = logreg.predict(Test_vect)

In [0]:
logreg.fit(X_vect,mbti['tactics'].values)
sub['tactics'] = logreg.predict(Test_vect)

In [0]:
sub.to_csv('subm.csv',index=False)